<a href="https://colab.research.google.com/github/distractedm1nd/mar2moon/blob/main/Wav2VecTrain_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

#NOTE
Really need to clean this file up, but has to wait a little bit... will focus on script for gathering the audio files from Tristans batch :)

In [2]:
from datasets import Dataset
from IPython.display import display, HTML
import pandas as pd
import re
import random

df = pd.read_csv("data.csv", sep=";", names=["Podcast", "Timestamp", "Timestamp(seconds)", "text", "keywords"])
df["file"] = df.apply(lambda x: f"output/{x.name}.wav", axis=1)
batch = Dataset.from_pandas(df).shuffle().train_test_split(test_size=0.3)
df.head()

,Podcast,Timestamp,Timestamp(seconds),text,keywords,file
0,LexFridman-Vitalik-2021-06-03,00:27,27.0,Bitcoin and many other cryptocurrencies have b...,"Bitcoin, Cryptocurrencies",output/0.wav
1,LexFridman-Vitalik-2021-06-03,17:40,1060.0,Mmm I definitely see Ethereum as uh being a me...,Ethereum,output/1.wav
2,LexFridman-Vitalik-2021-06-03,18:52,1132.0,I mentioned Ethereum challenges the power cent...,Ethereum,output/2.wav
3,LexFridman-Vitalik-2021-06-03,23:08,1388.0,If uh if cryptocurrency if Ethereum runs the w...,"Ethereum, cryptocurrency",output/3.wav
4,LexFridman-Vitalik-2021-06-03,40:47,2447.0,Miner extractable value as an existential risk...,"Ethereum, Miner, Value",output/4.wav


In [3]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

batch = batch.map(remove_special_characters)

In [4]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = batch.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=batch.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 17,
 "'": 9,
 'a': 4,
 'b': 26,
 'c': 6,
 'd': 7,
 'e': 0,
 'f': 24,
 'g': 11,
 'h': 14,
 'i': 23,
 'j': 25,
 'k': 15,
 'l': 13,
 'm': 27,
 'n': 21,
 'o': 22,
 'p': 20,
 'q': 8,
 'r': 12,
 's': 5,
 't': 2,
 'u': 3,
 'v': 19,
 'w': 16,
 'x': 18,
 'y': 1,
 'z': 10}

In [5]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [6]:
import json
with open('vocab2.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [7]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab2.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [8]:
#from pydub import AudioSegment
#for i in range(45):
#    sound = AudioSegment.from_wav(f"Data/audio/{i + 1}.wav")
#    sound = sound.set_channels(1)
#    sound = sound.set_frame_rate(16000)
#    sound.export(f"Data/audio/{i+1}.wav", format="wav")

In [9]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [10]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
processor.save_pretrained("wav2vec2-base-processor")

In [11]:
import soundfile as sf
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

batch = batch.map(speech_file_to_array_fn, remove_columns=batch.column_names["train"], num_proc=4)

In [12]:
import IPython.display as ipd
import numpy as np
import random


ipd.Audio(data=np.asarray(batch["train"][0]["speech"]), autoplay=False, rate=16000)

In [13]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

batch_prepared = batch.map(prepare_dataset, remove_columns=batch.column_names["train"], num_proc=4, batched=True, batch_size=8)

In [14]:
import torch
from datasets import load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}



In [15]:
pd.DataFrame(batch_prepared["train"])

,input_values,labels
0,"[-0.7773767100600254, -0.7573835922774863, -0....","[23, 17, 27, 0, 4, 21, 17, 2, 14, 23, 5, 17, 2..."
1,"[-0.000795677432079015, -0.001043837579625437,...","[27, 4, 1, 26, 0, 17, 3, 14, 17, 23, 2, 17, 16..."
2,"[-2.1726106311942783, -2.482006137276068, -2.8...","[4, 12, 0, 17, 7, 0, 24, 23, 21, 23, 2, 0, 13,..."
3,"[-0.10236011390578295, -0.11400223825391953, -...","[16, 14, 4, 2, 17, 23, 5, 17, 26, 23, 2, 6, 22..."
4,"[-0.004423142574036325, -0.007182175913669025,...","[5, 22, 17, 24, 23, 12, 5, 2, 17, 16, 23, 2, 1..."
5,"[0.02812148930639354, 0.029186888988953108, 0....","[26, 23, 2, 6, 22, 23, 21, 17, 4, 21, 7, 17, 2..."
6,"[-0.00012454511854935669, -0.00012454511854935...","[23, 24, 17, 1, 22, 3, 17, 6, 22, 27, 20, 4, 1..."
7,"[0.6009296296102777, 0.5216712259052105, 0.363...","[0, 2, 14, 0, 12, 0, 3, 27, 17, 5, 27, 4, 12, ..."
8,"[-1.0961715241311416, -1.2244125340006577, -1....","[23, 17, 27, 0, 21, 2, 23, 22, 21, 0, 7, 17, 0..."
9,"[-0.2944924343915673, -0.3156161964259577, -0....","[4, 21, 7, 17, 23, 21, 17, 27, 4, 21, 1, 17, 1..."


In [20]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [21]:
model.freeze_feature_extractor()

In [23]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir="wav2vec2-test",
  group_by_length=False,
  per_device_train_batch_size=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  learning_rate=1e-4,
  weight_decay=0.005,
  save_steps=500,
  eval_steps=10,
  logging_steps=10,
  warmup_steps=100,
  save_total_limit=2,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=batch_prepared["train"],
    eval_dataset=batch_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [24]:
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
10,8.658600,7.050540,1.000000,3.548900,3.663000
20,6.000100,3.514814,1.000000,3.638700,3.573000
30,3.838600,3.613131,1.000000,3.649200,3.562000
40,4.090600,2.968771,1.000000,3.595300,3.616000
50,3.355100,2.930973,1.000000,3.550500,3.661000
60,3.188800,2.999514,1.000000,3.583700,3.627000
70,3.170800,2.919139,1.000000,3.547500,3.665000
80,3.042700,2.899749,1.000000,3.545100,3.667000
90,3.008200,2.893930,1.000000,3.510700,3.703000
100,2.957900,2.915578,1.000000,3.497200,3.717000


TrainOutput(global_step=420, training_loss=3.257205259232294, metrics={'train_runtime': 743.4652, 'train_samples_per_second': 0.565, 'total_flos': 2.543867359488e+17, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 3858919, 'init_mem_gpu_alloc_delta': 1091793408, 'init_mem_cpu_peaked_delta': 47953445, 'init_mem_gpu_peaked_delta': 12020980224, 'train_mem_cpu_alloc_delta': 680654, 'train_mem_gpu_alloc_delta': 3781134336, 'train_mem_cpu_peaked_delta': 51970435, 'train_mem_gpu_peaked_delta': 1064014848})

In [30]:
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits


    pred_ids = torch.argmax(logits, dim=-1)

    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

results = batch.map(map_to_result)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
pd.DataFrame(results["train"])

,pred_str,sampling_rate,speech,target_text
0,THIS IS THE END OF ETHERIUM AND I THINK A LOT ...,16000,"[-0.09014892578125, -0.08782958984375, -0.0875...",i mean this is the end of ethereum and i think...
1,MAYBE A IT WON'T BE BIT COIN THERE PLACES GOLD...,16000,"[-9.1552734375e-05, -0.0001220703125, 0.0, 6.1...",maybe uh it won't be bitcoin that replaces gol...
2,RE DEFINITELY CASES WHEREI COULD THINK THE ETH...,16000,"[-0.2198486328125, -0.25115966796875, -0.28933...",are definitely cases where like i think the et...
3,WHAT IS BICKWEN WISE AND IMPORTANT TO THE WORL...,16000,"[-0.01177978515625, -0.01312255859375, -0.0149...",what is bitcoin and why is it important to the...
4,SO FIRST WITH BID COIN AND THEN WITH DOSH COIN...,16000,"[-0.000457763671875, -0.000762939453125, -0.00...",so first with bitcoin and then with dogecoin s...
5,BITCOIN AND MANY OTHER CRIPLIC CURRENCIES HAVE...,16000,"[0.00323486328125, 0.00335693359375, 0.0032043...",bitcoin and many other cryptocurrencies have b...
6,IF YOU COMPARE BICKPOINT TO THE DOLLAR THERE'S...,16000,"[0.0, 0.0, -6.103515625e-05, -3.0517578125e-05...",if you compare bitcoin to the dollar there's a...
7,RIUM SMART CONTRACTS KIN JUST RUN ARBITRARY CO...,16000,"[0.0643310546875, 0.05584716796875, 0.03890991...",ethereum smart contracts can just run arbitrar...
8,AMENTION DETHERIUM CHALLENGES H THE THE POWER ...,16000,"[-0.105682373046875, -0.1180419921875, -0.1144...",i mentioned ethereum challenges the power cent...
9,E MOST ENTERTAINING CRIPTAL CURRENCY AS CURPTO...,16000,"[-0.032318115234375, -0.034637451171875, -0.03...",and in many ways dogecoin is the most entertai...


In [32]:
model.to("cuda")
input_values = processor(batch["test"][0]["speech"], sampling_rate=batch["test"][0]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
    logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

"<pad> <pad> <pad> <pad> <pad> <pad> I <pad> | | <pad> <pad> T H H I N N <pad> K <pad> <pad> | | | | <pad> <pad> <pad> <pad> <pad> <pad> B I <pad> <pad> <pad> G <pad> | | <pad> C <pad> O <pad> <pad> I <pad> N <pad> <pad> | | <pad> I <pad> S | | | A | | <pad> <pad> <pad> R <pad> E <pad> <pad> <pad> V <pad> O <pad> L <pad> U <pad> <pad> T I O O N <pad> <pad> <pad> | | I <pad> <pad> <pad> <pad> D <pad> E <pad> <pad> <pad> <pad> A <pad> <pad> <pad> <pad> <pad> | <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> N <pad> <pad> <pad> <pad> <pad> <pad> | <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> O <pad> N <pad> | | <pad> <pad> M A <pad> <pad> N <pad> <pad> Y | | | | <pad> <pad> F <pad> R O <pad> <pad> <pad> N <pad> <pad> T <pad> <pad> S <pad> | | | <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> B U <pad> T <pad> | <pad> <pad> <pad> S <pad> <pad> T <pad> I I <pad> L <pad> <pad> <pad>